In [59]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [124]:
class Value:
  
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"
  
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
    
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
    
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
    
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
      
        return out
  
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out
  
    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
    
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
    
        return out
  
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
    
        def _backward():
            self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
        out._backward = _backward
    
        return out
  
  
    def backward(self):
    
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
    
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [125]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b
n.label = 'n'
o = n.tanh()
o.label = 'o'

In [126]:
o.backward()

In [127]:
x2.grad

0.4999999999999999

In [128]:
a = Value(4.0)
b = Value(3.0)
a / b

Value(data=1.3333333333333333)

In [214]:
class Neuron:
  
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
    # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params

class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        # print(nin, nouts, sz)
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [215]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.17760343742033416)

In [216]:
n.parameters()

[Value(data=-0.0201706529315282),
 Value(data=0.6249623723004125),
 Value(data=0.9515915501320218),
 Value(data=-0.32978621710412437),
 Value(data=0.6625249415813539),
 Value(data=-0.5259576897623104),
 Value(data=-0.28512352143129704),
 Value(data=-0.19716278774844764),
 Value(data=-0.2804922751284056),
 Value(data=0.9610865212131467),
 Value(data=0.8592282463409884),
 Value(data=0.01307166937727633),
 Value(data=0.4021686155396973),
 Value(data=-0.05013139735547645),
 Value(data=0.9058633519976502),
 Value(data=0.7678899138831812),
 Value(data=0.4543382104426126),
 Value(data=-0.32604227674437003),
 Value(data=-0.01955112005452464),
 Value(data=-0.0475221589370054),
 Value(data=-0.8848058834361678),
 Value(data=-0.8027430598009357),
 Value(data=-0.6731359494843223),
 Value(data=-0.22349347861759483),
 Value(data=0.7848021173071966),
 Value(data=-0.3888860264584313),
 Value(data=-0.39301119544447416),
 Value(data=-0.454132188918805),
 Value(data=-0.5526424542639858),
 Value(data=0.020

In [217]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]

In [224]:
for k in range(10):
    
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.01 * p.grad

    print(k, loss.data)

0 0.1648364364755553
1 0.15885345775592152
2 0.15325582585740466
3 0.14800846958654268
4 0.14308040343283646
5 0.13844415172803406
6 0.13407526743178233
7 0.1299519278851475
8 0.12605459354231202
9 0.12236571852390879


In [225]:
ypred

[Value(data=0.8774622338360707),
 Value(data=-0.8134714859085665),
 Value(data=-0.8093507744599072),
 Value(data=0.8097102193514646)]